In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Synthetic Data Generation Use Case with Vertex AI and Snowfakery

This notebook provides a comprehensive solution for generating synthetic data using Vertex AI Gemini generative model and the Snowfakery data generation tool.
It leverages the power of these technologies to create realistic and customizable synthetic datasets based on existing data schemas and user inputs.

## Key Features

* **Schema Extraction:** Extracts table schemas from BigQuery to understand the structure and data types of your source data.
* **Sample Data Analysis:** Analyzes sample data from BigQuery tables to inform the generation of Snowfakery recipes.
* **Automated Recipe Generation:** Uses Gemini to automatically generate Snowfakery recipes based on table schemas and sample data.
* **Recipe Refinement:** Provides prompts and tools for refining and correcting generated recipes, including handling relationships between tables.
* **Data Generation with Snowfakery:** Utilizes Snowfakery to generate synthetic data based on the refined recipes and user-specified counts.
* **Data Storage and Loading:** Uploads generated data to Google Cloud Storage (GCS) and loads it into target BigQuery tables.

## Workflow

1. **Installation and Setup:**
   - Install the required packages: Vertex AI SDK and Snowfakery.
   - Restart the runtime to ensure the newly installed packages are available.
   - Authenticate your notebook environment if using Google Colab.
   - Set your Google Cloud project ID and location, and initialize the Vertex AI SDK.

2. **Data Extraction and Analysis:**
   - Specify the source BigQuery tables for data extraction.
   - Define the number of sample rows to fetch from each table.
   - Extract the schema of each source table.
   - Read sample data from the source tables and upload it to GCS.

3. **Recipe Generation and Refinement:**
   - Use Gemini to generate initial Snowfakery recipes based on the extracted schemas and sample data.
   - Refine the recipes using prompts and tools to handle data types, relationships, and constraints.
   - Correct any syntax errors or inconsistencies in the recipes.

4. **Data Generation and Loading:**
   - Specify the desired number of records to generate for each table.
   - Use Snowfakery to generate synthetic data based on the refined recipes and specified counts.
   - Upload the generated data to GCS.
   - Specify the target BigQuery tables for loading the synthetic data.
   - Load the data from GCS into the target BigQuery tables.

## Customization Options

* **Source and Target Tables:** Modify the `source_table_list` and `target_table_list` parameters to specify your desired tables.
* **GCS Bucket:** Change the `gcs_bucket_name` parameter to use your own GCS bucket.
* **Sample Data Size:** Adjust the `total_rows_to_fetch_from_source_table` parameter to control the number of sample rows used for recipe generation.
* **Generated Data Counts:** Update the `user_counts` dictionary to specify the desired number of records for each table.
* **Fake Function List:** Modify the `fake_function_list` to include or exclude specific fake functions for data generation.
* **Prompts:** Customize the prompts used for recipe generation, correction, and relationship building to tailor the process to your needs.

## Benefits

* **Data Privacy:** Generate synthetic data that resembles real data without exposing sensitive information.
* **Data Augmentation:** Increase the size of your datasets for improved model training and testing.
* **Data Diversity:** Create synthetic data with specific characteristics or distributions to address imbalances or biases in your original data.
* **Data Sharing:** Share synthetic data with collaborators or external parties without compromising privacy.

## Conclusion

This notebook provides a powerful and flexible solution for generating synthetic data. By combining the capabilities of Vertex AI's Gemini and Snowfakery, you can create high-quality synthetic datasets that meet your specific needs and contribute to more robust and ethical data analysis and machine learning.

# Synthetic Data Generation Usecase

| | |
|-|-|
| Author | [Anas Aslam](https://github.com/aaslam17)

## Getting Started

### Install Vertex AI SDK and other required packages

In [ ]:
%pip install --upgrade --user -q google-cloud-aiplatform snowfakery==3.6.3

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id="project_id")

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "project_id"  # @param {type:"string"}
LOCATION = "region_id"  # @param {type:"string"}

import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [ ]:
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd
from datetime import datetime
import io
import google.api_core.exceptions
from vertexai.generative_models import GenerativeModel,GenerationConfig,Part
from snowfakery.plugins import SnowfakeryPlugin
from snowfakery import generate_data
from io import StringIO
from IPython.display import display, Markdown
import yaml
import json

#Get the user inputs

In [ ]:
# Semicolon-separated list of source BigQuery tables for retrieving sample data
source_table_list="project_id.dataset_id.table_id1;project_id.dataset_id.table_id2;project_id.dataset_id.table_id3"

# Semicolon-separated list of target BigQuery tables for loading generated data.
target_table_list="project_id.dataset_id.table_id1_dest;project_id.dataset_id.table_id2_dest;project_id.dataset_id.table_id3_dest"

# Number of rows to fetch from each source table for analysis and recipe generation.
total_rows_to_fetch_from_source_table=30

# Name of the Google Cloud Storage bucket for storing intermediate and output data.
gcs_bucket_name="gcs_bucket_name"

# Dictionary specifying the desired number of records to generate for each table.
user_counts = {
    "table_id1": 5,
    "table_id2": 10,
    "table_id3": 50
}

#Read the source table , extract sample records and upload to GCS Bucket

In [ ]:
def read_table_and_upload_to_gcs(project_id, dataset_id, table_id, bucket_name,blob_name,row_limit):
    """
    Reads from a BigQuery table and uploads to GCS.

    Args:
        project_id: Your Google Cloud project ID.
        dataset_id: The BigQuery dataset ID.
        table_id: The BigQuery table ID.
        bucket_name: The name of your GCS bucket.
        blob_name: The name of the blob (file) to create in the GCS bucket.
        row_limit: The maximum number of rows to read from the table.
    """

    # Construct the table reference string
    table_ref = f"{project_id}.{dataset_id}.{table_id}"

    # Establish a BigQuery client connection
    bq_client = bigquery.Client(project=project_id)

    # Create a query job with the row limit
    query_job = bq_client.query(f"SELECT * FROM `{table_ref}` ORDER BY RAND() LIMIT {row_limit}")

    # Get query results as a pandas DataFrame
    df = query_job.to_dataframe()

    # Create a GCS client
    storage_client = storage.Client()

    # Get a reference to the GCS bucket
    bucket = storage_client.bucket(bucket_name)

    # Create a blob (file) in the bucket
    blob = bucket.blob(blob_name)

    # Upload the DataFrame as a CSV file to GCS
    blob.upload_from_string(df.to_csv(index=False), content_type="text/csv")

    print(f"Uploaded {len(df)} rows to gs://{bucket_name}/{blob_name}")


# Read the Source Table and Extract the Table Schema

In [ ]:
def extract_schema(project,dataset_id,table_id):
    """
    Extracts the schema of a BigQuery table as a JSON string.

    Args:
        project: The ID of the Google Cloud project.
        dataset_id: The ID of the BigQuery dataset.
        table_id: The ID of the BigQuery table.

    Returns:
        A JSON string representing the table schema.
    """

    client = bigquery.Client()  # Initialize BigQuery client
    dataset_ref = client.dataset(dataset_id, project=project)  # Get dataset reference
    table_ref = dataset_ref.table(table_id)  # Get table reference
    table = client.get_table(table_ref)  # Get table object

    f = io.StringIO("")  # Create a string buffer
    client.schema_to_json(table.schema, f)  # Write schema to buffer as JSON
    return f.getvalue()  # Return the JSON string

#Generate Synthetic Data based on the Input Recipe

In [ ]:
def generate_data_with_user_input(recipe):
    """
    Generates synthetic data using Snowfakery based on user input.

    Args:
        recipe: The Snowfakery recipe as a string.
        output_format: The desired output format (e.g., csv, json).
        output_folder: The path to the folder where output will be saved.
    """

    # Generate data using Snowfakery
    generate_data(
        StringIO(recipe),
        output_format="csv",
        output_folder="/output"
    )

    print("Data generation complete. Check the output folder.")



#Upload the generated data file from local to GCS bucket

In [ ]:
def upload_from_local_to_gcs(bucket_name, blob_name, local_file_path):
    """
    Uploads a CSV file fron local storage to Google Cloud Storage.

    Args:
        bucket_name: The name of the GCS bucket.
        blob_name: The name of the blob (file) in GCS.
        file_path: The local path to the CSV file.
    """

    # Initialize GCS client
    client = storage.Client()

    # Get a reference to the bucket
    bucket = client.bucket(bucket_name)

    # Create a blob object
    blob = bucket.blob(blob_name)

    # Read the CSV file into a Pandas DataFrame
    df = pd.read_csv(local_file_path)

    # Upload the DataFrame as a CSV file to GCS
    blob.upload_from_string(df.to_csv(index=False), content_type="text/csv")

    print(f"Uploaded local file to gs://{bucket_name}/{blob_name}")
    print(f"The uploaded file contains "+str(len(df))+ " records")

#Load the GCS File to Target Bigquery Table

In [ ]:
def copy_table_schema(source_project_id,dest_project_id,source_dataset_id,dest_dataset_id, source_table_id,dest_table_id,):
    """
    Creates a new Target BigQuery table with the same schema as an Source table.

    Args:
        source_project_id: Your Source Google Cloud project ID.
        dest_project_id: Your Target Google Cloud project ID.
        source_dataset_id: The dataset ID of the source table.
        source_table_id: The table ID of the source table.
        dest_dataset_id: The dataset ID where the new table will be created.
        dest_table_id: The desired table ID for the new table.
    """

    # Construct table reference strings
    source_table_ref = f"{source_project_id}.{source_dataset_id}.{source_table_id}"
    dest_table_ref = f"{dest_project_id}.{dest_dataset_id}.{dest_table_id}"

    # Establish a BigQuery client connection
    bq_client = bigquery.Client()

    # Get the source table's information
    source_table = bq_client.get_table(source_table_ref)

    # Create a new table with the same schema (fields)
    table = bigquery.Table(dest_table_ref, schema=source_table.schema)
    bq_client.create_table(table)

    print(f"Created table {dest_table_ref} with the same schema as {source_table_ref}")

def gcs_to_bigquery(source_project_id,dest_project_id,source_dataset_id,dest_dataset_id, source_table_id,dest_table_id, bucket_name, file_name):
    """
    Reads the receipe output from GCS bucket and loads it into a BigQuery table.
    Create the destination table, if the table doesnot exist

    Args:
        source_project_id: Your Source Google Cloud project ID.
        dest_project_id: Your Target Google Cloud project ID.
        source_dataset_id: The dataset ID of the source table.
        source_table_id: The table ID of the source table.
        dest_dataset_id: The dataset ID of the target table
        dest_table_id: The table ID of the target table
        bucket_name: The name of the GCS bucket.
        blob_name: The name of the output file to load to bigquery
    """
    # Construct the table reference string
    dest_table_ref = f"{dest_project_id}.{dest_dataset_id}.{dest_table_id}"

    # Establish a BigQuery client connection
    bq_client = bigquery.Client(project=source_project_id)

    try:
        bq_client.get_table(dest_table_ref)
        print(f"Table {dest_table_id} exists in dataset {dest_dataset_id}")
    except google.api_core.exceptions.NotFound:
        copy_table_schema(source_project_id,dest_project_id,source_dataset_id,dest_dataset_id, source_table_id,dest_table_id)

    #blob_name=f"{dest_project_id}/{dest_dataset_id}/{dest_table_id}/output/"+file_name;
    blob_name=file_name;

    # Get a reference to the GCS bucket and blob
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # Configure the BigQuery job
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,  # Adjust if your file is not CSV
        skip_leading_rows=1,  # Skip header row (if present)
        autodetect=False, # Autodetect schema
        ignore_unknown_values = True #Ignore the id column generated by snowfakery
    )

    # Create and execute the load job
    load_job = bq_client.load_table_from_uri(
        blob.public_url, dest_table_ref, job_config=job_config
    )

    load_job.result()  # Wait for the job to complete

    print(f"Loaded data from gs://{bucket_name}/{blob_name} to {dest_table_ref}")



#List of fake functions to be used

In [ ]:
fake_function_list="""
fake: Email
fake: Name
fake: Postalcode
fake: State
Address Fakers
fake: City
fake: Country
fake: CurrentCountry
fake: StreetAddress
Company Fakers
fake: CatchPhrase
fake: Company
Date_Time Fakers
fake: Time
fake: Year
Lorem Fakers
fake: Paragraph
fake: Sentence
fake: Text
fake: Word
Person Fakers
fake: FirstName
fake: FirstNameFemale
fake: FirstNameMale
fake: FirstNameNonbinary
fake: LastName
Phone_Number Fakers
fake: PhoneNumber
Rarely Used
Salesforce Fakers
fake: UserName
fake: Alias
fake: RealisticMaybeRealEmail
Address Fakers
fake: Address
fake: AdministrativeUnit
fake: BuildingNumber
fake: CityPrefix
fake: CitySuffix
fake: CountryCode
fake: CurrentCountryCode
fake: MilitaryApo
fake: MilitaryDpo
fake: MilitaryShip
fake: MilitaryState
fake: PostalcodeInState
fake: PostalcodePlus4
fake: Postcode
fake: PostcodeInState
fake: SecondaryAddress
fake: StateAbbr
fake: StreetName
fake: StreetSuffix
fake: Zipcode
fake: ZipcodeInState
fake: ZipcodePlus4
Automotive Fakers
fake: LicensePlate
Bank Fakers
fake: Aba
fake: BankCountry
fake: Bban
fake: Iban
fake: Swift
fake: Swift11
fake: Swift8
Barcode Fakers
fake: Ean
fake: Ean13
fake: Ean8
fake: LocalizedEan
fake: LocalizedEan13
fake: LocalizedEan8
fake: UpcA
fake: UpcE
Color Fakers
fake: Color
fake: ColorName
fake: HexColor
fake: RgbColor
fake: RgbCssColor
fake: SafeColorName
fake: SafeHexColor
Company Fakers
fake: Bs
fake: CompanySuffix
Credit_Card Fakers
fake: CreditCardExpire
fake: CreditCardFull
fake: CreditCardNumber
fake: CreditCardProvider
fake: CreditCardSecurityCode
Currency Fakers
fake: Cryptocurrency
fake: CryptocurrencyCode
fake: CryptocurrencyName
fake: Currency
fake: CurrencyCode
fake: CurrencyName
fake: CurrencySymbol
fake: Pricetag
Date_Time Fakers
fake: AmPm
fake: Century
fake.DateBetween:
fake: DateObject
fake: DateOfBirth
fake: DayOfMonth
fake: DayOfWeek
fake: FutureDate
fake: FutureDatetime
fake: Iso8601
fake: Month
fake: MonthName
fake: PastDate
fake: PastDatetime
fake: Pytimezone
fake: TimeDelta
fake: TimeObject
fake: TimeSeries
fake: Timezone
fake: UnixTime
Decorators.Py Fakers
fake: AsciiCompanyEmail
fake: AsciiEmail
fake: AsciiFreeEmail
fake: AsciiSafeEmail
fake: CompanyEmail
fake: DomainName
fake: DomainWord
fake: FreeEmail
fake: FreeEmailDomain
fake: Hostname
fake: SafeDomainName
fake: SafeEmail
fake: Slug
File Fakers
fake: FileExtension
fake: FileName
fake: FilePath
fake: MimeType
fake: UnixDevice
fake: UnixPartition
Geo Fakers
fake: Coordinate
fake: Latitude
fake: Latlng
fake: LocalLatlng
fake: LocationOnLand
fake: Longitude
Internet Fakers
fake: Dga
fake: HttpMethod
fake: IanaId
fake: ImageUrl
fake: Ipv4
fake: Ipv4NetworkClass
fake: Ipv4Private
fake: Ipv4Public
fake: Ipv6
fake: MacAddress
fake: PortNumber
fake: RipeId
fake: Tld
fake: Uri
fake: UriExtension
fake: UriPage
fake: UriPath
fake: Url
Isbn Fakers
fake: Isbn10
fake: Isbn13
Job Fakers
fake: Job
Lorem Fakers
fake: Paragraphs
fake: Sentences
fake: Texts
fake: Words
Misc Fakers
fake: Binary
fake: Boolean
fake: Csv
fake: Dsv
fake: FixedWidth
fake: Image
fake: Json
fake: Md5
fake: NullBoolean
fake: Password
fake: Psv
fake: Sha1
fake: Sha256
fake: Tar
fake: Tsv
fake: Uuid4
fake: Zip
Person Fakers
fake: LanguageName
fake: LastNameFemale
fake: LastNameMale
fake: LastNameNonbinary
fake: Name
fake: NameFemale
fake: NameMale
fake: NameNonbinary
fake: Prefix
fake: PrefixFemale
fake: PrefixMale
fake: PrefixNonbinary
fake: Suffix
fake: SuffixFemale
fake: SuffixMale
fake: SuffixNonbinary
Phone_Number Fakers
fake: CountryCallingCode
fake: Msisdn
Profile Fakers
fake: Profile
fake: SimpleProfile
Providers Fakers
fake: Bothify
fake: Hexify
fake: LanguageCode
fake: Lexify
fake: Locale
fake: Numerify
Python Fakers
fake: Pybool
fake: Pydecimal
fake: Pydict
fake: Pyfloat
fake: Pyint
fake: Pyiterable
fake: Pylist
fake: Pyset
fake: Pystr
fake: PystrFormat
fake: Pystruct
fake: Pytuple
Ssn Fakers
fake: Ein
fake: InvalidSsn
fake: Itin
fake: Ssn
User_Agent Fakers
fake: AndroidPlatformToken
fake: Chrome
fake: Firefox
fake: InternetExplorer
fake: IosPlatformToken
fake: LinuxPlatformToken
fake: LinuxProcessor
fake: MacPlatformToken
fake: MacProcessor
fake: Opera
fake: Safari
fake: UserAgent
fake: WindowsPlatformToken
"""

#Prompt for Recipe Creation

In [ ]:
def create_receipe_prompt(fake_function_list,table_schema):
  Recipe_Prompt= f"""
Purpose and Goals:

* Assist users in creating Snowfakery recipes based on their table schemas.
* Generate recipes that accurately reflect the structure and data types of the input schemas.
* Prioritize clarity and readability in the generated recipes.

Behaviors and Rules:

1. Schema Extraction:
 - Carefully parse the JSON schema provided by the user.
 - Extract relevant information for each column, including 'name', 'type', and 'description'.

2. Recipe Generation:
 - Analyze the extracted column information.
 - Consult the Snowfakery documentation ([https://snowfakery.readthedocs.io/en/docs/fakedata.html](https://snowfakery.readthedocs.io/en/docs/fakedata.html)) to understand about each fake function usage
 - Use the below fake function list to determine appropriate fake functions for each column based on its type and description. {fake_function_list}
 - Construct a Snowfakery recipe in YAML format, using the identified fake functions and incorporating any relationships between tables.
  - For the fields similar to indentifier field, use ${{unique_id}}

3. Receipe Validation:
 - Carefully read the user provided sample records and for each of the string type column, provide random_choice as arguments for each of the recipe fields from the sample records.
 - For integer, float, deciaml, bigdecimal fields, use the random_number with min and max values
 - Use random_choice and random_number, only if no fake function can be found for those fields. If a fake function is found for a particular field, stick to that fake function only, instead of random_choice and random_number
 - If there is any column dependencies based on the semantic meaning of the columns. Example Start_Date depends on End_Date, Start_time depends on End_Time.
 - Apply the below technique to create column dependencies. Make sure the indentations and colons are kept in the same way
 - Strictly validate the number of curly braces used. Example it should $ sign followed by have two ${{unique_id}}.
 - Strictly keep the provided table name as the yaml object name

Example: Column dependencies between travel_start_date and travel_end_date, trip_start_datetime and trip_end_datetime ,random_choice for name field, id field and random_number is as below
- object: trips
  count: 5
  fields:
  trip_id:${{unique_id}}
  trip_type:
   random_choice:
    - One Way
    - Round Trip
  name:
   fake: Name
  email:
   fake: Email
  age:
   random_number:
    min: 12
    max: 95
  travel_start_date:
  fake.DateBetween:
   start_date: -365d
   end_date: -1d
 travel_end_date:
 fake.DateBetween:
  start_date: +0d
  end_date: +365d
 - For Date and Datetime Column Types, strictly use "fake.DateBetween:" as mentioned in the example above.
 - Since fake function is available for the name and email field, respective fake function is used.
 - Since fake fuction is not avaialble for the trip_type and age, random_choice and random_number is used.

-For any column which is semantic closer to the Identifier. Example Trip_Id, Plan_Id etc, always use the unique_id Faker function.

- Make sure you follow the same output pattern as above without missing any colon, indentation, hypen, line breaks etc
- Strictly include only the below list of fake functions

 {fake_function_list}

 4. Recipe Presentation:

 - Present the generated Snowfakery recipe to the user in well formatted YAML format only.
 - Include comments in the recipe to explain any complex or non-obvious choices.
 - Remind the user that the generated recipe is a starting point and encourage them to customize and refine it as needed.

 5. Validation:
 - Validate that the recipe doesnot inclue any fake funciton which is not part of the list shared earlier.
 - Validate that the random_choice is correctly added for all the string type fields
 - Strictly validate that the output is a valid YAML with no special characters or formatting issues.
 - Strictly validate the number of curly braces used. Example it should $ sign followed by have two curly braces like ${{unique_id}}.
 - Strictly Keep the same indentation and line breaks as provided in the example recipe.
 - Strictly keep object name same as the table_name in the provided input schema


Additional Notes:
* Strictly adhere to the fake functions defined in the Snowfakery documentation and the provided list. If a faker function is not in the provided list, do not include it in the recipe.
* Strictly format the output in the valid YAML format with the correct formatting
* Strictly Keep the same indentation and line breaks as provided in the example recipe.
* Remove ```yaml and ``` from the output
* The output should contain only the receip and no other explanation


User Schmea Input:

{table_schema}

"""
  return Recipe_Prompt;


#Prompt for Recipe Correction

In [ ]:
yaml_correction_prompt="""You are a YAML text processing expert. Your task is to carefully analyze a given YAML text and modify it according to the following rule:

Rule 1: If a field value within the YAML text contains only a single pair of curly braces (e.g., `${field}`), replace it with two pairs of curly braces (e.g., `${{field}}`).
Rule 2: Convert the decimal values in the min and max arguments of the random_number function to the corresponding integer values

**Input:**

The user will provide you with a string of YAML text.

**Output:**

**Updated YAML:**
1. Return the modified YAML text where all single curly brace expressions have been replaced with double curly braces.
2. Return the modified YAML with the decmail fields replaced with the corresponding integer values

**Important Considerations:**

* **YAML Syntax:** Ensure that your modifications maintain the correct YAML syntax.
* **Preservation:**  Preserve the structure and indentation of the original YAML.
* **Clarity:** Make your output clear and easy to understand.

**Example:**

If the user provides the following YAML:

- object: employees
  count: 5
  fields:
    sales_id: ${unique_id}
    sales_name:
      random_choice:
        - Henry Jones
        - Ivy Taylor
        - Kelly White
    tax:
      random_number:
        min: 21.87
        max: 48.98
    win_id: ${unique_id}

Your Output should be
- object: employees
  count: 5
  fields:
    sales_id: ${{unique_id}} --Added extra curly braces
    sales_name:
      random_choice:
        - Henry Jones
        - Ivy Taylor
        - Kelly White
    tax:
      random_number:
        min: 21 -- Converted from Decimal to Integer
        max: 48 -- Converted from Decimal to Integer
    win_id: ${{unique_id}} --Added extra curly braces

* Strictly Keep the same indentation and line breaks as provided in the example recipe.
* Strictly keep the same object name as in the input
* Remove ```yaml and ``` from the output
* The output should contain only the receip and no other explanation
"""


#Prompt for building the relationship between the tables

In [ ]:
Relationship_builder="""Purpose: Given the schema of multiple database tables, determine their relationships and the type of each table.



Input Format:

*  A string containing the table definitions. Each definition includes:

  *  Table name.

  *  Column names and their data types.



Output Format:

* A JSON array where each element represents a table and contains:

  *  table_name (string): The name of the table.

  *  table_type (string): Either 'dimension' or 'fact'.

  *  foreign_keys (array):

    *  column_name (string): The name of the foreign key column in this table.

    *  references_table (string): The name of the referenced table.

    *  references_column (string): The name of the referenced column.

    *  relationship_type (string): Either 'One-to-Many (1:M)' or 'Many-to-One (M:1)'. If the table is a dimension table, this array is empty.



Rules and Instructions:

1. **Schema Parsing:**

   *  Identify each table, its name, columns, and data types.



2. **Table Type Determination:**

   *  Classify each table as either:

      *  **Dimension:** Primarily stores descriptive attributes, often used to filter or group data in fact tables.

      *  **Fact:** Primarily stores measurable or quantitative data, often related to events or transactions.

   *  If unsure, make an educated guess based on common patterns



3. **Relationship Identification:**

   *  Look for foreign key relationships between tables.

      *  Foreign keys are columns that reference the primary key of another table.

   *  Determine the type of relationship:

      *  **One-to-Many (1:M):** One record in the dimension table can be associated with many records in the fact table.

      *  **Many-to-One (M:1):** Many records in the fact table can be associated with one record in the dimension table.



4. **Output Generation:**

   *  For each table, create a JSON object following the specified format.

   *  Include relevant foreign key information in the `foreign_keys` array.

   *  Ensure the output is valid JSON.


Additional Notes:

*  This is a simplified approach. Real-world schema analysis can be more complex.

*  For large schemas, you might implement more advanced algorithms for relationship discovery.
"""

#Prompt for updating the Recipe based on the table relationship

In [ ]:
Relationship_Prompt="""Below is the Example of Snowfakery yml file for tables emp and dept with minimal number of columns to demonstrate how primary key and foreign key can be represented while creating recipe.
- object: dept
  count: 4
  fields:
    deptno: ${{unique_id}}
    name:
     random_choice:
       - Henry Jones
       - Ivy Taylor
       - Kelly White
- object: emp
  count: 10
  fields:
    __dept_fk:
      random_reference:
        to: dept
        unique: False
    empno: ${{unique_id}}
    deptno: ${{__dept_fk.deptno}}


## Snowfakery Recipe Breakdown: Department and Employee Data Generation

This Snowfakery recipe generates synthetic data for two related objects: `dept` (department) and `emp` (employee). It leverages several key features of Snowfakery to create realistic and interconnected data.

**Object: dept**

- **`object: dept`**: This line declares the first object to be generated, representing a department.
- **`count: 4`**: This specifies that four department records will be created.
- **`fields:`**: This introduces the fields that will be populated for each department record.
    - **`deptno: ${{unique_id}}`**: This assigns a unique ID to each department using the `unique_id` function, ensuring each department has a distinct identifier.
    - **`name: fake: Name`**: This generates a fake department name using the `fake` function with the `Name` provider. This will produce random, yet plausible, department names.

**Object: emp**

- **`object: emp`**: This line declares the second object to be generated, representing an employee.
- **`count: 10`**: This specifies that ten employee records will be created.
- **`fields:`**: This introduces the fields that will be populated for each employee record.
    - **`__dept_fk: random_reference: to: dept unique: False`**: This is a hidden field used to establish a relationship between employees and departments. It utilizes the `random_reference` function to randomly assign a department to each employee.
        - **`to: dept`**: This specifies that the reference should point to a `dept` object.
        - **`unique: False`**: This allows multiple employees to be assigned to the same department, reflecting a realistic scenario.
    - **`empno: ${{unique_id}}`**: This assigns a unique ID to each employee using the `unique_id` function, ensuring each employee has a distinct identifier.
    - **`deptno: ${{__dept_fk.deptno}}`**: This field populates the employee's department number by referencing the `deptno` field of the randomly assigned department from the hidden `__dept_fk` field. This creates a direct link between the employee and their department.

**Recipe Execution**

When this recipe is executed, Snowfakery will first generate four unique department records with random names. Subsequently, it will create ten employee records, each randomly assigned to one of the existing departments. The `deptno` field in the `emp` object will reflect the `deptno` of the assigned department, ensuring data consistency and reflecting a realistic one-to-many relationship between departments and employees.

**Key Features Utilized**

- **`unique_id` function**: Generates unique identifiers for both departments and employees.
- **`fake` function**: Generates fake department names using the `Name` provider.
- **`random_reference` function**: Randomly assigns departments to employees.
- **Hidden fields**: Used to store intermediate values and establish relationships without appearing in the final output.

This recipe demonstrates the power and flexibility of Snowfakery in generating synthetic data with realistic relationships between objects. It can be easily adapted to generate data for other scenarios by modifying the object names, field definitions, and relationships as needed.


Task:
* Correct the below provided Snowfakery recipe (yml) files to include correct primary key generation and foreign key references.
* Use the provided relationship
* Remove ```yaml and ``` from the output
* The output should contain only the receip and no other explanation
* Strictly Keep the same indentation and line breaks as provided in the example recipe.
* Strictly keep the provided table name as the yaml object name


Here's a step-by-step guide for completing your tasks:
1. For recipes having ONLY PRIMARY KEY include unique_id as shown in the dept, emp example recipe.
2. For recipes having FOREIGN KEY reference include
a. Hidden field referring to Primary table recipe
PS: unique=False for 'One-to-Many (1:M)' relationship and unique=True for 'One-to-One (1:1)' relationship
b. Hidden fields should be placed right after `fields` keyword of that particular object
c. Replace the foreign key column value to refer to primary column recipe
3. When there are conflicting scenarios between
4. All other columns will remain as it is. No changes are required for other columns.
5. You are allowed to rearrange the order of yaml recipe to get desired output (hint: try building independent objects i.e., without foreign key first and only after that bring in dependent objects)
6. Strictly Keep the same indentation and line breaks as provided in the example recipe.
7. Strictly keep the same object name as in the input
8. Always add - before the "object" keyword in the yaml recipe
9. Just provide output yaml; no explaination is needed
"""

#Prompt for correcting the object name in the Recipe

In [ ]:
Correct_Object_Name_Prompt=""" Your task is to read the relationship input between tables, extract the table name form the relationship.
Then, read the provided yaml receipe and replace the object name with the name of the table names extracted from the relationship.
* Just provide output yaml; no explaination is needed
* Remove ```yaml and ``` from the output
*  Always add - before the "object" keyword in the yaml recipe
* Strictly Keep the same indentation and line breaks as provided in the input recipe.
"""

### Initialize model

Initialize the Gemini model with the desired model parameters and `Tool` that we defined earlier:

In [ ]:
gemini_model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0)
)

In [ ]:
def generate_receipes(source_table_list,gcs_bucket_name,total_rows_to_fetch_from_source_table):
    """
    Generates Snowfakery recipes for a list of BigQuery source tables.

    Args:
        source_table_list: A semicolon-separated list of source BigQuery tables.
        gcs_bucket_name: The name of the GCS bucket for storing data.
        total_rows_to_fetch_from_source_table: The number of rows to fetch from each source table.

    Returns:
        A tuple containing the combined YAML recipe and the combined schema.
    """
    source_tables = source_table_list.split(';')
    combined_yaml=''
    combined_schema=''
    for src_table in source_tables:
        source_table = src_table.split('.')
        source_project_id = source_table[0]
        source_dataset_id = source_table[1]
        source_table_id = source_table[2]
        table_schema=extract_schema(source_project_id,source_dataset_id,source_table_id)
        combined_schema=combined_schema+"Table Name:"+source_table_id+table_schema
        Recipe_Prompt=create_receipe_prompt(fake_function_list,table_schema)
        current_timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
        source_file_name = f"{source_project_id}/{source_dataset_id}/{source_table_id}/input/extract_{current_timestamp}.csv"
        read_table_and_upload_to_gcs(source_project_id, source_dataset_id, source_table_id, gcs_bucket_name, source_file_name, total_rows_to_fetch_from_source_table)
        source_file_path = f"gs://{gcs_bucket_name}/{source_file_name}"
        bigquery_records = Part.from_uri(source_file_path, mime_type="text/csv")
        yaml_output = gemini_model.generate_content([Recipe_Prompt, "Sample Records:", bigquery_records]).text
        chat_yaml_corection = gemini_model.start_chat()
        final_yaml_output=chat_yaml_corection.send_message(yaml_correction_prompt + """ **Input:** """ + yaml_output).text
        combined_yaml=combined_yaml+final_yaml_output
    return combined_yaml,combined_schema

In [ ]:
def generate_output_data(source_table_list,gcs_bucket_name,total_rows_to_fetch_from_source_table):
        """
        Generates synthetic data and loads it into BigQuery tables.

        This function orchestrates the process of generating synthetic data
        based on source BigQuery tables and loading it into target BigQuery tables.
        It involves generating Snowfakery recipes, extracting table relationships,
        creating and uploading data to GCS, and finally loading the data into BigQuery.
        Args:
          source_table_list: A semicolon-separated list of source BigQuery tables.
          gcs_bucket_name: The name of the GCS bucket for storing data.
          total_rows_to_fetch_from_source_table: The number of rows to fetch from each source table.

          Returns:
          None
        """
        combined_yaml,combined_schema=generate_receipes(source_table_list,gcs_bucket_name,total_rows_to_fetch_from_source_table)
        relationship_output= gemini_model.generate_content([Relationship_builder,combined_schema])
        print(relationship_output.text)
        final_yaml_output = gemini_model.generate_content([Relationship_Prompt,relationship_output.text,combined_yaml])
        corrected_yaml_output = gemini_model.generate_content([Correct_Object_Name_Prompt,relationship_output.text,final_yaml_output.text])
        recipe = corrected_yaml_output.text
        # Load the YAML recipe
        yaml_data = yaml.safe_load(recipe)

        # Update the counts for each object
        for item in yaml_data:
          object_name = item.get('object')
          if object_name in user_counts:
            item['count'] = user_counts[object_name]

        # Dump the updated YAML recipe
        updated_recipe = yaml.dump(yaml_data, sort_keys=False)
        print(updated_recipe)
        generate_data_with_user_input(updated_recipe)

        source_tables = source_table_list.split(';')
        target_tables = target_table_list.split(';')

        # Create a dictionary to map source tables to target tables
        table_map = {}

        # Iterate through the lists and create the mapping
        for i in range(len(source_tables)):
            source_table_name = source_tables[i]  # Extract table name from fully qualified name
            target_table_name = target_tables[i]
            table_map[source_table_name] = target_table_name

        # Load the JSON data
        data = json.loads(json.dumps(table_map))

        # Iterate through the key-value pairs
        for key, value in data.items():
            source_table_name=key.split('.')[-1]
            source_dataset_name=key.split('.')[-2]
            source_project_id=key.split('.')[-3]
            target_table_name=value.split('.')[-1]
            target_dataset_name=value.split('.')[-2]
            target_project_id=value.split('.')[-3]
            now = datetime.now()
            current_timestamp = now.strftime("%Y%m%d%H%M%S%f")
            target_file_name=f"{target_project_id}/{target_dataset_name}/{target_table_name}/output/generated_{current_timestamp}.csv"
            local_file_path=f"/output/{source_table_name}.csv"
            upload_from_local_to_gcs(gcs_bucket_name, target_file_name, local_file_path)
            gcs_to_bigquery(source_project_id,target_project_id,source_dataset_name,target_dataset_name, source_table_name,target_table_name, gcs_bucket_name, target_file_name)
        return None


In [ ]:
#Generate Synthetic Data by calling the below function
generate_output_data(source_table_list,gcs_bucket_name,total_rows_to_fetch_from_source_table)